<a href="https://colab.research.google.com/github/RKiddle/Python-Play/blob/main/Tiny_Thai_Lessons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U google-generativeai gTTS gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.5 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from gtts import gTTS
import gradio as gr
import os
from google.colab import userdata
from datetime import datetime # Import datetime for unique filenames

# --- SETUP ---
# Get your API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Use a lightweight, fast model
model = genai.GenerativeModel('gemini-pro-latest') # Changed model to gemini-pro-latest

def generate_lesson(scenario, lang_code):
    """Generates a Tiny Lesson based on a specific scenario and language code."""

    # Map language codes to human-readable names for the prompt
    lang_map = {
        "th": "Thai",
        "zh-cn": "Chinese (Mandarin)",
        "ja": "Japanese",
        "ko": "Korean"
    }
    lang_name = lang_map.get(lang_code, lang_code) # Default to code if not found (shouldn't happen with dropdown)

    # 1. The Prompt: We tell Gemini strictly how to format the lesson
    prompt = f"""
    Act as a friendly {lang_name} language tutor. I want a 'Tiny Lesson' about: "{scenario}".

    Please provide the output in this exact structure:
    1. **Core Vocabulary**: 5 essential {lang_name} words for this scenario ({lang_name} Script - Romanization - English Meaning).
    2. **Key Phrase**: 1 useful sentence for this situation.
    3. **Grammar Tip**: A very brief, one-sentence tip related to the phrase.
    4. **Cultural Note**: A tiny fun fact about {lang_name} culture related to this.

    Keep it bite-sized and encouraging.
    """

    try:
        response = model.generate_content(prompt)
        lesson_text = response.text

        # Generate unique filenames for saving
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        text_output_filename = f"lesson_text_{timestamp}.md"
        audio_output_filename = f"lesson_audio_{timestamp}.mp3"

        # 2. Save the lesson text to a file
        with open(text_output_filename, "w", encoding="utf-8") as f:
            f.write(lesson_text)

        # 3. Generate the audio file using the provided language code
        tts = gTTS(lesson_text, lang=lang_code)
        tts.save(audio_output_filename)

        return lesson_text, audio_output_filename, text_output_filename

    except Exception as e:
        return f"Error: {str(e)}", None, None

# --- THE INTERFACE ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🌍 Little Language Lesson Generator")
    gr.Markdown("Type a scenario below and select a language.")

    with gr.Row():
        inp = gr.Textbox(label="What do you want to learn today?", placeholder="e.g., Ordering spicy soup")
        lang_dropdown = gr.Dropdown(
            [
                ("Thai", "th"),
                ("Chinese (Mandarin)", "zh-cn"),
                ("Japanese", "ja"),
                ("Korean", "ko")
            ],
            value="th", # Default to Thai
            label="Select Language"
        )
        btn = gr.Button("Generate Lesson", variant="primary")

    out_text = gr.Markdown(label="Your Lesson")
    out_audio = gr.Audio(label="Listen")
    out_text_file = gr.File(label="Download Lesson Text (.md)", type="filepath") # Added for text download

    btn.click(fn=generate_lesson, inputs=[inp, lang_dropdown], outputs=[out_text, out_audio, out_text_file])

    gr.Markdown("<div style='text-align: center; margin-top: 20px;'>© 2025 Richard Kiddle</div>")

# Launch the app
app.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://166f51bd8f219be3fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
